In [1]:
import numpy as np
from pathlib import Path
import sys

In [2]:
import sys
import ctypes
import numpy as np
from ctypes.util import find_library

class mgard:

    def __init__(self,mgard_path = None):

        """
        init mgard
        :parmmgard library path
        """

        # if mgard_path ==None:
        #     mgard_path = {
        #         "darwin": "libmgardpy.dylib",
        #         "windows": "libmgardpy.dll",
        #     }.get(sys.platform, "libmgardpy.so")

        self.mgard = ctypes.CDLL(mgard_path)
        
        """
        float * compress_decompress(float * data, int N, size_t* dims, float eb, float s, size_t& compressed_size)
        """
        self.mgard.compress_decompress_float.argtypes =[
            np.ctypeslib.ndpointer(dtype=np.float32),
            ctypes.c_int,
            np.ctypeslib.ndpointer(dtype=np.uint64),
            ctypes.c_float,
            ctypes.c_float,
            ctypes.POINTER(ctypes.c_size_t)
        ]
        # ndpointer(dtype=np.float32, shape=(3,))
        self.mgard.compress_decompress_float.restype = ctypes.POINTER(ctypes.c_float)

        self.libc = ctypes.CDLL(ctypes.util.find_library('c'))
        self.libc.free.argtypes = (ctypes.c_void_p,)
        
    def compress(self, data, eb, s):
        """
        compress_decompress(float * data, int N, size_t* dims, float eb, float s, size_t& compressed_size)
        """
        dims = np.asarray(data.shape).astype(np.uint64)
        N = ctypes.c_int(len(dims))
        compressed_size = ctypes.c_size_t()
        decompressed_data = self.mgard.compress_decompress_float(data, N, dims, eb, s, ctypes.byref(compressed_size))
        # ddata = np.empty(compressed_size.value, dtype=np.float32)
        dd = np.ctypeslib.as_array(decompressed_data, shape=data.shape).copy()
        # dd = np.array(decompressed_data[:np.prod(data.shape)]).reshape(data.shape)
        ratio = (data.size * data.itemsize)/ float(compressed_size.value)
        self.libc.free(decompressed_data)
        print(ratio)
        # ddata = np.array(decompressed_data[:np.prod(data.shape)]).reshape(data.shape)
        return dd, ratio
    
    def verify(self, src_data, dec_data):
        """
        Compare the decompressed data with original data
        :param src_data: original data, numpy array
        :param dec_data: decompressed data, numpy array
        :return: max_diff, psnr, nrmse
        """
        data_range = np.max(src_data) - np.min(src_data)
        diff = src_data - dec_data
        max_diff = np.max(abs(diff))
        print("abs err={:.8G}".format(max_diff))
        mse = np.mean(diff ** 2)
        nrmse = np.sqrt(mse) / data_range
        psnr = 20 * np.log10(data_range) - 10 * np.log10(mse)
        return max_diff, psnr, nrmse
        

In [2]:
import sys
import ctypes
import numpy as np
from ctypes.util import find_library

class mgard:

    def __init__(self,mgard_path = None):

        """
        init mgard
        :parmmgard library path
        """

        # if mgard_path ==None:
        #     mgard_path = {
        #         "darwin": "libmgardpy.dylib",
        #         "windows": "libmgardpy.dll",
        #     }.get(sys.platform, "libmgardpy.so")

        self.mgard = ctypes.CDLL(mgard_path)
        
        """
        float * compress_decompress(float * data, int N, size_t* dims, float eb, float s, size_t& compressed_size)
        """
        self.mgard.compress_decompress.argtypes =[ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float), ctypes.c_int, 
                                                  ctypes.POINTER(ctypes.c_size_t), ctypes.c_float, 
                                                  ctypes.c_float, ctypes.POINTER(ctypes.c_size_t)]

        # ndpointer(dtype=np.float32, shape=(3,))
        # self.mgard.compress_decompress_float.restype = ctypes.POINTER(ctypes.c_float)
        
    def compress(self, data, eb, s):
        """
        compress_decompress(float * data, float* ddata, int N, size_t* dims, float eb, float s, size_t& compressed_size)
        """
        data_ptr = data.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
        ddata = np.empty(data.size, dtype=np.float32)
        ddata_ptr = ddata.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
        dims = np.asarray(data.shape).astype(np.uint64).ctypes.data_as(ctypes.POINTER(ctypes.c_size_t))
        N = ctypes.c_int(len(data.shape))
        compressed_size = ctypes.c_size_t()
        self.mgard.compress_decompress_float(data_ptr, ddata_ptr, N, dims, eb, s, ctypes.byref(compressed_size))

        ratio = (data.size * data.itemsize)/ float(compressed_size.value)
        print(ratio)
        return ddata, ratio
    
    def verify(self, src_data, dec_data):
        """
        Compare the decompressed data with original data
        :param src_data: original data, numpy array
        :param dec_data: decompressed data, numpy array
        :return: max_diff, psnr, nrmse
        """
        data_range = np.max(src_data) - np.min(src_data)
        diff = src_data - dec_data
        max_diff = np.max(abs(diff))
        print("abs err={:.8G}".format(max_diff))
        mse = np.mean(diff ** 2)
        nrmse = np.sqrt(mse) / data_range
        psnr = 20 * np.log10(data_range) - 10 * np.log10(mse)
        return max_diff, psnr, nrmse
        

In [3]:
MGARD = mgard("/home/jp/git/MGARD/examples/python/build/libmgardpy.so")

In [4]:
cldhgh = np.fromfile("/home/jp/data/cesm/CLDHGH_1_1800_3600.dat", dtype=np.float32).reshape(1800, 3600)

In [5]:
ddata, ratrio= MGARD.compress(cldhgh, 0.5, 0)

ArgumentError: argument 5: <class 'TypeError'>: Don't know how to convert parameter 5

In [ ]:
ddata

: 

: 